# Pre-process ERA5 pressure level data for CREDIT

This notebook provides methods on gathering ERA5 pressure level data from NCAR/RDA and ARCO-ERA5. The RDA data requires internal access of the glade file system at NCAR.

## Data preparation

* **Pressure-level analysis (RDA)**
    * geopotential, u_component_of_wind, v_component_of_wind, temperature, specific_humidity
* **Single-level analysis (RDA)**
    * surface_pressure, mean_sea_level_pressure
    * sea_surface_temperature, skin_temperature, 2m_temperature,
    * 10m_u_component_of_wind, 10m_v_component_of_wind, total_cloud_cover
* **Single-level forecasts (ARCO)**
    * total_precipitation, evaporation
    * top_net_solar_radiation, top_net_thermal_radiation
    * surface_net_solar_radiation, surface_net_thermal_radiation, surface_latent_heat_flux, surface_sensible_heat_flux

**References**

* NCAR/RDA
    * [ERA5 Reanalysis (0.25 Degree Latitude-Longitude Grid)](https://rda.ucar.edu/datasets/d633000/)
    * glade storage: `/glade/campaign/collections/rda/data/d633000/`
* ARCO-ERA5
    * [Google Cloud storage](https://console.cloud.google.com/storage/browser/gcp-public-data-arco-era5)
    * [Project page at GitHub](https://github.com/google-research/arco-era5)
    * Complete hourly file: `gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3`



In [1]:
import os
import sys
import yaml
import dask
import zarr
import numpy as np
import xarray as xr
import pandas as pd
from glob import glob
from dask.utils import SerializableLock

import calendar
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

sys.path.insert(0, os.path.realpath('../libs/'))
import verif_utils as vu

In [2]:
# import multiprocessing
# from dask.distributed import Client
# from dask_jobqueue import PBSCluster

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

## Upper-air variables

**NCAR/RDA**

An example of file listing at `/glade/campaign/collections/rda/data/d633000/e5.oper.an.pl/197901`
```
e5.oper.an.pl.128_060_pv.ll025sc.1979010100_1979010123.nc    
e5.oper.an.pl.128_132_v.ll025uv.1979010100_1979010123.nc   
e5.oper.an.pl.128_203_o3.ll025sc.1979010100_1979010123.nc
e5.oper.an.pl.128_075_crwc.ll025sc.1979010100_1979010123.nc  
e5.oper.an.pl.128_133_q.ll025sc.1979010100_1979010123.nc
e5.oper.an.pl.128_246_clwc.ll025sc.1979010100_1979010123.nc
e5.oper.an.pl.128_076_cswc.ll025sc.1979010100_1979010123.nc  
e5.oper.an.pl.128_135_w.ll025sc.1979010100_1979010123.nc   
e5.oper.an.pl.128_247_ciwc.ll025sc.1979010100_1979010123.nc
e5.oper.an.pl.128_129_z.ll025sc.1979010100_1979010123.nc     
e5.oper.an.pl.128_138_vo.ll025sc.1979010100_1979010123.nc  
e5.oper.an.pl.128_248_cc.ll025sc.1979010100_1979010123.nc
e5.oper.an.pl.128_130_t.ll025sc.1979010100_1979010123.nc     
e5.oper.an.pl.128_155_d.ll025sc.1979010100_1979010123.nc
e5.oper.an.pl.128_131_u.ll025uv.1979010100_1979010123.nc     
e5.oper.an.pl.128_157_r.ll025sc.1979010100_1979010123.nc
```

In [4]:
year = 2000
N_days = 366 if year % 4 == 0 else 365

config_name = os.path.realpath('data_config_6h.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

In [5]:
compress = zarr.Blosc(cname='zstd', clevel=1, shuffle=zarr.Blosc.SHUFFLE, blocksize=0)


chunk_size_4d = dict(chunks=(conf['RDA']['chunk_size_4d']['time'],
                                 conf['RDA']['chunk_size_4d']['level'],
                                 conf['RDA']['chunk_size_4d']['latitude'],
                                 conf['RDA']['chunk_size_4d']['longitude']))

dict_encoding = {}

for i_var, var in enumerate(conf['RDA']['varname_upper_air']):
    dict_encoding[var] = {'compressor': compress, **chunk_size_4d}

In [6]:
# increase the file cache size
xr.set_options(file_cache_maxsize=500)
# lock for safe parallel access
netcdf_lock = SerializableLock()

# all days within a year
start_time = datetime(year, 1, 1, 0, 0)
dt_list = [start_time + timedelta(days=i) for i in range(N_days)]

# upper-air var names
varnames = list(conf['RDA']['varname_upper_air'].values())

ds_list = []

for i_day, dt in enumerate(dt_list):
    # file source info
    base_dir = dt.strftime(conf['RDA']['source']['anpl_format'])
    dt_pattern = dt.strftime(conf['RDA']['source']['anpl_dt_pattern_format'])

    # get upper-air vars
    filename_collection = [glob(base_dir + f'*{var}*{dt_pattern}*')[0] for var in varnames]
    
    if len(filename_collection) != len(varnames):
        raise ValueError(f'Year {year}, day {day_idx} has incomplete files')
    
    # Open with a lock to avoid race conditions when accessing files
    ds = xr.open_mfdataset(filename_collection, combine='by_coords', parallel=True, lock=netcdf_lock)

    # drop useless var
    ds = ds.drop_vars('utc_date', errors='ignore')

    # hourly --> 6 hourly
    ds = ds.isel(time=slice(0, -1, 6))
    
    #  chunking
    ds = ds.chunk(conf['RDA']['chunk_size_4d'])
    ds_list.append(ds)
    
# concatenate
ds_yearly = xr.concat(ds_list, dim='time')

# save to zarr
base_dir = conf['RDA']['save_loc'] + 'upper_air/' 
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

save_name = base_dir + conf['RDA']['prefix'] + '_upper_air_{}.zarr'.format(year)

# ds_yearly.to_zarr(save_name, mode='w', consolidated=True, compute=True, encoding=dict_encoding)

### Single-level variables

An example of file listing at `/glade/campaign/collections/rda/data/d633000/e5.oper.an.sfc/197901`

```
e5.oper.an.sfc.128_015_aluvp.ll025sc.1979010100_1979013123.nc  e5.oper.an.sfc.128_167_2t.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_016_aluvd.ll025sc.1979010100_1979013123.nc  e5.oper.an.sfc.128_168_2d.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_017_alnip.ll025sc.1979010100_1979013123.nc  e5.oper.an.sfc.128_170_stl2.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_018_alnid.ll025sc.1979010100_1979013123.nc  e5.oper.an.sfc.128_183_stl3.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_031_ci.ll025sc.1979010100_1979013123.nc     e5.oper.an.sfc.128_186_lcc.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_032_asn.ll025sc.1979010100_1979013123.nc    e5.oper.an.sfc.128_187_mcc.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_033_rsn.ll025sc.1979010100_1979013123.nc    e5.oper.an.sfc.128_188_hcc.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_034_sstk.ll025sc.1979010100_1979013123.nc   e5.oper.an.sfc.128_198_src.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_035_istl1.ll025sc.1979010100_1979013123.nc  e5.oper.an.sfc.128_206_tco3.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_036_istl2.ll025sc.1979010100_1979013123.nc  e5.oper.an.sfc.128_229_iews.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_037_istl3.ll025sc.1979010100_1979013123.nc  e5.oper.an.sfc.128_230_inss.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_038_istl4.ll025sc.1979010100_1979013123.nc  e5.oper.an.sfc.128_231_ishf.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_039_swvl1.ll025sc.1979010100_1979013123.nc  e5.oper.an.sfc.128_232_ie.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_040_swvl2.ll025sc.1979010100_1979013123.nc  e5.oper.an.sfc.128_235_skt.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_041_swvl3.ll025sc.1979010100_1979013123.nc  e5.oper.an.sfc.128_236_stl4.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_042_swvl4.ll025sc.1979010100_1979013123.nc  e5.oper.an.sfc.128_238_tsn.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_059_cape.ll025sc.1979010100_1979013123.nc   e5.oper.an.sfc.128_243_fal.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_066_lailv.ll025sc.1979010100_1979013123.nc  e5.oper.an.sfc.128_244_fsr.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_067_laihv.ll025sc.1979010100_1979013123.nc  e5.oper.an.sfc.128_245_flsr.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_078_tclw.ll025sc.1979010100_1979013123.nc   e5.oper.an.sfc.228_008_lmlt.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_079_tciw.ll025sc.1979010100_1979013123.nc   e5.oper.an.sfc.228_009_lmld.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_134_sp.ll025sc.1979010100_1979013123.nc     e5.oper.an.sfc.228_010_lblt.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_136_tcw.ll025sc.1979010100_1979013123.nc    e5.oper.an.sfc.228_011_ltlt.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_137_tcwv.ll025sc.1979010100_1979013123.nc   e5.oper.an.sfc.228_012_lshf.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_139_stl1.ll025sc.1979010100_1979013123.nc   e5.oper.an.sfc.228_013_lict.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_141_sd.ll025sc.1979010100_1979013123.nc     e5.oper.an.sfc.228_014_licd.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_148_chnk.ll025sc.1979010100_1979013123.nc   e5.oper.an.sfc.228_089_tcrw.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_151_msl.ll025sc.1979010100_1979013123.nc    e5.oper.an.sfc.228_090_tcsw.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_159_blh.ll025sc.1979010100_1979013123.nc    e5.oper.an.sfc.228_131_u10n.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_164_tcc.ll025sc.1979010100_1979013123.nc    e5.oper.an.sfc.228_132_v10n.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_165_10u.ll025sc.1979010100_1979013123.nc    e5.oper.an.sfc.228_246_100u.ll025sc.1979010100_1979013123.nc
e5.oper.an.sfc.128_166_10v.ll025sc.1979010100_1979013123.nc    e5.oper.an.sfc.228_247_100v.ll025sc.1979010100_1979013123.nc
```

In [7]:
year = 2000
N_months = 12

config_name = os.path.realpath('data_config_6h.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

In [8]:
compress = zarr.Blosc(cname='zstd', clevel=1, shuffle=zarr.Blosc.SHUFFLE, blocksize=0)


chunk_size_3d = dict(chunks=(conf['RDA']['chunk_size_3d']['time'],
                             conf['RDA']['chunk_size_3d']['latitude'],
                             conf['RDA']['chunk_size_3d']['longitude']))

dict_encoding = {}

for i_var, var in enumerate(conf['RDA']['varname_single']):
    dict_encoding[var] = {'compressor': compress, **chunk_size_3d}

In [9]:
# increase the file cache size
xr.set_options(file_cache_maxsize=500)
# lock for safe parallel access
netcdf_lock = SerializableLock()

# all days within a year
start_time = datetime(year, 1, 1, 0, 0)
dt_list = [start_time + relativedelta(months=i) for i in range(N_months)]

In [10]:
# var names
varnames = list(conf['RDA']['varname_single'].values())

ds_list = []

for i_mon, dt in enumerate(dt_list):
    # file source info
    base_dir = dt.strftime(conf['RDA']['source']['ansfc_format'])

    first_day = datetime(year, dt.month, 1)
    last_day = datetime(year, dt.month, calendar.monthrange(year, dt.month)[1])
    
    dt_pattern = dt.strftime(conf['RDA']['source']['ansfc_dt_pattern_format'])
    dt_pattern = dt_pattern.format(first_day.day, last_day.day)
    
    # get upper-air vars
    filename_collection = [glob(base_dir + f'*{var}*{dt_pattern}*')[0] for var in varnames]
    
    if len(filename_collection) != len(varnames):
        raise ValueError(f'Year {year}, day {day_idx} has incomplete files')
    
    # Open with a lock to avoid race conditions when accessing files
    ds = xr.open_mfdataset(filename_collection, combine='by_coords', parallel=True, lock=netcdf_lock)

    # drop useless var
    ds = ds.drop_vars('utc_date', errors='ignore')

    # hourly --> 6 hourly
    ds = ds.isel(time=slice(0, -1, 6))
    
    #  chunking
    ds = ds.chunk(conf['RDA']['chunk_size_3d'])
    ds_list.append(ds)
    
# concatenate
ds_yearly = xr.concat(ds_list, dim='time')

# save to zarr
base_dir = conf['RDA']['save_loc'] + 'surf/' 
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

save_name = base_dir + conf['RDA']['prefix'] + '_surf_{}.zarr'.format(year)

# ds_yearly.to_zarr(save_name, mode='w', consolidated=True, compute=True, encoding=dict_encoding)

## Single-level accumulative variables

**RDA**

* Example file listing: `/glade/campaign/collections/rda/data/d633000/e5.oper.fc.sfc.accumu/197901`

**ARCO**

* Hourly data from `gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3`
* Accumulate hourly to 6 hourly: `xarray.resample(time='6h').sum()`

In [11]:
year = 1979
N_days = 366 if year % 4 == 0 else 365

config_name = os.path.realpath('data_config_6h.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

In [12]:
# save to zarr
base_dir = conf['ARCO']['save_loc'] + 'accum/' 
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

compress = zarr.Blosc(cname='zstd', clevel=1, shuffle=zarr.Blosc.SHUFFLE, blocksize=0)


chunk_size_3d = dict(chunks=(conf['ARCO']['chunk_size_3d']['time'],
                             conf['ARCO']['chunk_size_3d']['latitude'],
                             conf['ARCO']['chunk_size_3d']['longitude']))

dict_encoding = {}

for i_var, var in enumerate(conf['ARCO']['varname_accum']):
    dict_encoding[var] = {'compressor': compress, **chunk_size_3d}

In [13]:
ERA5_1h = xr.open_zarr(
    "gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3",
    chunks=None,
    storage_options=dict(token='anon'),)

time_start = '{}-12-31T00'.format(year-1) # hourly accum var needs one extra day to accum on 6 hrs
time_start_save = '{}-01-01T00'.format(year)
time_end = '{}-12-31T23'.format(year)
ERA5_1h_yearly = ERA5_1h.sel(time=slice(time_start, time_end))

variables_levels = {}
for varname in conf['ARCO']['varname_accum']:
    variables_levels[varname] = None

ERA5_1h_save = vu.ds_subset_everything(ERA5_1h_yearly, variables_levels)

ERA5_1h_shifted = ERA5_1h_save.shift(time=-1)
ERA5_6h = ERA5_1h_shifted.resample(time='6h').sum()
ERA5_6h['time'] = ERA5_6h['time'] + pd.Timedelta(hours=6)

ERA5_6h_save = ERA5_6h.sel(time=slice(time_start_save, time_end))

ERA5_6h_save = ERA5_6h_save.chunk(conf['ARCO']['chunk_size_3d'])
save_name = base_dir + conf['ARCO']['prefix'] + '_accum_{}.zarr'.format(year)
# ERA5_6h_save.to_zarr(save_name, mode="w", consolidated=True, compute=True, encoding=dict_encoding)

**Comparing accumulated hourly to the old directly available 6 hourly data**

In [34]:
# ERA5_6h = xr.open_zarr(
#     "gs://gcp-public-data-arco-era5/ar/1959-2022-6h-1440x721.zarr",
#     chunks=None,
#     storage_options=dict(token='anon'),)

# tp_6h_ref = ERA5_6h['total_precipitation_6hr']
# tp_6h_ref = tp_6h_ref.sel(time=slice(time_start_save, time_end))

In [24]:
# np.sum(np.array(ERA5_6h_save['total_precipitation'].isel(time=-1)) - np.array(tp_6h_ref.isel(time=-1)))

0.0

### Static variables

## Others

### Total precipitation from RDA


```python
xr_ARCO = xr.open_zarr('/glade/derecho/scratch/ksha/CREDIT_data/ERA5_plevel_base/test_data/surf_test.zarr')
tp_ARCO = xr_ARCO['total_precipitation']

base_dir = '/glade/campaign/collections/rda/data/d633000/e5.oper.fc.sfc.accumu/197901/'
xr_RDA_CP = xr.open_dataset(base_dir+'e5.oper.fc.sfc.accumu.128_143_cp.ll025sc.1979010106_1979011606.nc')
xr_RDA_LP = xr.open_dataset(base_dir+'e5.oper.fc.sfc.accumu.128_142_lsp.ll025sc.1979010106_1979011606.nc')

xr_RDA_CP = xr_RDA_CP.drop_vars('utc_date', errors='ignore')
xr_RDA_CP = xr_RDA_CP.rename({'CP': 'TP'})
xr_RDA_LP = xr_RDA_LP.drop_vars('utc_date', errors='ignore')
xr_RDA_LP = xr_RDA_LP.rename({'LSP': 'TP'})

da = xr_RDA_CP + xr_RDA_LP

time_deltas = pd.to_timedelta(da["forecast_hour"].values, unit="h")
new_times = np.add.outer(da["forecast_initial_time"].values, time_deltas)
new_times = new_times.flatten()

da_an = da.stack(time=("forecast_initial_time", "forecast_hour"))
da_an = da_an.drop_vars(['forecast_hour', 'forecast_initial_time', 'time'])
da_an = da_an.assign_coords(time=new_times)

for i_hour in range(10):
    # i + 7 becuase ini_time = 06Z, fcst_lead_time starts from 01 hr
    tp_ARCO_np = np.array(tp_ARCO.isel(time=i_hour+7))
    da_np = np.array(da_an['TP'].isel(time=i_hour))
    print(np.sum(np.abs(tp_ARCO_np - da_np)))

# ARCO vs. RDA
data_var = da['TP']
tp_RDA = data_var.isel(forecast_initial_time=0)
tp_RDA_np = np.array(tp_RDA)
tp_ARCO_np = np.array(tp_ARCO.isel(time=slice(7, 7+12)))
np.sum(np.abs(tp_ARCO_np[3, ...] - tp_RDA_np[3, ...]))
```